In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import pyrender
import matplotlib.pyplot as plt
import numpy as np

from face_reconstruction.optim import NearestNeighborMode, DistanceType, run_icp, run_icp_combined, BFMOptimization, \
    KeyframeOptimizationParameters, run_icp_keyframes, BFMOptimizationParameters
from face_reconstruction.pipeline import BFMPreprocessor
from face_reconstruction.plots import PlotManager, plot_params
from face_reconstruction.graphics import draw_pixels_to_image
from face_reconstruction.landmarks import detect_landmarks

In [ ]:
frame_id = 36 # 69: smiling
              # 86: o

# 1. Parameters

## 1.1. Sparse Reconstruction params

In [ ]:
n_params_shape_sparse = 30
n_params_expression_sparse = 30
weight_shape_params_sparse = 1000
weight_expression_params_sparse = 1000
l2_regularization_sparse = 1000

## 1.2. Dense Reconstruction params

In [ ]:
nn_mode = NearestNeighborMode.FACE_VERTICES  # FACE_VERTICES: every face vertex will be assigned its nearest neighbor in pointcloud
# POINTCLOUD: every point in pointcloud will be assigned its nearest neighbor in face model
distance_type = DistanceType.POINT_TO_POINT
icp_iterations = 2
optimization_steps_per_iteration = 10
l2_regularization_dense = 4000  # 10000 for Lie

n_params_shape_dense = 30
n_params_expression_dense = 30
weight_shape_params_dense = 100  # 10000, 10000000000 for POINT_TO_PLANE
weight_expression_params_dense = 100  # 1000, 10000000000 for POINT_TO_PLANE

weight_sparse_term = 10

# 2. Preprocessing

In [ ]:
preprocessor = BFMPreprocessor()

In [ ]:
img, depth_img, intrinsics = preprocessor.load_frame(frame_id)
pointcloud, pointcloud_normals, colors = preprocessor.to_3d(img, depth_img, intrinsics)
landmark_points, bfm_landmark_indices, face_pointcloud, face_pointcloud_colors = preprocessor.detect_landmarks(img, depth_img, intrinsics, threshold_landmark_deviation=200, ignore_jawline=True)

# 3. Face Reconstruction

## 3.1. Sparse Reconstruction

In [ ]:
sparse_optimizer = BFMOptimization(preprocessor.bfm,
                                   n_params_shape=n_params_shape_sparse,
                                   n_params_expression=n_params_expression_sparse,
                                   weight_shape_params=weight_shape_params_sparse,
                                   weight_expression_params=weight_expression_params_sparse,
                                   rotation_mode='lie')

In [ ]:
initial_params = preprocessor.get_initial_params(sparse_optimizer)

In [ ]:
sparse_loss = sparse_optimizer.create_sparse_loss_3d(bfm_landmark_indices, landmark_points, regularization_strength=l2_regularization_sparse)
sparse_context = sparse_optimizer.create_optimization_context(sparse_loss, initial_params)
result = sparse_context.run_optimization(sparse_loss, initial_params)
params_sparse = sparse_context.create_parameters_from_theta(result.x)

## 3.2. Dense Reconstruction

In [ ]:
dense_optimizer = BFMOptimization(preprocessor.bfm,
                                  n_params_shape=n_params_shape_dense,
                                  n_params_expression=n_params_expression_dense,
                                  weight_shape_params=weight_shape_params_dense,
                                  weight_expression_params=weight_expression_params_dense,
                                  rotation_mode='lie')

In [ ]:
params_dense, distances, dense_param_history = run_icp_combined(dense_optimizer,
                                                                bfm_landmark_indices,
                                                                landmark_points,
                                                                 face_pointcloud,
                                                                 preprocessor.bfm,
                                                                 params_sparse.with_new_manager(dense_optimizer),
                                                                 max_iterations=icp_iterations,
                                                                 nearest_neighbor_mode=nn_mode,
                                                                 distance_type=distance_type,
                                                                 max_nfev=optimization_steps_per_iteration,
                                                                 l2_regularization=l2_regularization_dense,
                                                                 pointcloud_normals=pointcloud_normals,
                                                               weight_sparse_term=weight_sparse_term)

# 4. Plots

In [ ]:
params_plot = params_dense

In [ ]:
scene = preprocessor.setup_scene(params_plot, show_pointcloud_face=False, show_landmarks=True, show_pointcloud=False)

In [ ]:
pyrender.Viewer(scene, use_raymond_lighting=True, viewport_size=(preprocessor.img_width, preprocessor.img_height))

In [ ]:
plot_params(params_plot.expression_coefficients)

In [ ]:
plot_params(params_plot.shape_coefficients)

In [ ]:
landmarks_img, face_pos = detect_landmarks(preprocessor.img, return_face_pos=True)
img_with_landmarks = np.array(preprocessor.img)
draw_pixels_to_image(img_with_landmarks, landmarks_img, color=[0, 255, 0])

In [ ]:
plt.imshow(img_with_landmarks)

In [ ]:
plt.figure(figsize=(12, 8))
plt.imshow(preprocessor.render_onto_img(params_plot, show_landmarks=True))

In [ ]:
error = preprocessor.plot_reconstruction_error(params_plot)
plt.xlabel(f"Mean Reconstruction Error: {error:.3f}")

In [ ]:
plot_manager = PlotManager.new_run("3d_dense_reconstruction")
preprocessor.store_param_history(plot_manager, '', dense_param_history)
plot_manager.generate_video('iteration_', '.jpg')